In [0]:
!pip3 install torch torchvision

    100% |████████████████████████████████| 519.5MB 29kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5a0cc000 @  0x7f31d7a422a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 61kB 13.8MB/s 
    100% |████████████████████████████████| 2.0MB 4.4MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Specify path to files on Google Drive

ml_path = '/content/gdrive/My Drive/ML2018/'

In [0]:
# add Google Drive path to system path for imports

import sys
sys.path.insert(0, ml_path)

In [17]:
# List files in Google Drive

import os
os.listdir(ml_path)

['neg_train.txt',
 'pos_train.txt',
 'vocab_pretrained_reduced.pkl',
 'embeddings200_pretrained_reduced.npy',
 'helpers.py',
 '__pycache__',
 'test_data.txt',
 'train_neg_full.txt',
 'train_pos_full.txt',
 'cnn1_reduced_32.csv',
 'cnns.py',
 'vocab_pretrained.pkl',
 'embeddings200.npy',
 'network.pt',
 'cnn1_reduced_128.csv',
 'tweets_pos_wstp.txt',
 'tweets_neg_wstp.txt',
 'test_data_wstp.txt',
 'vocab_pretrained_wstp_reduced.pkl',
 'embeddings200_wstp_pretrained_reduced.npy',
 'network_1_128_full.pt',
 'cnn1_128_reduced_wstp.csv']

In [34]:
import numpy as np
import pickle
import time

import torch
from torch import optim
from torch.autograd import Variable
import torch.utils.data as utils

from helpers import *
from cnns import *


# define file paths
path_embeddings = ml_path + 'embeddings200_pretrained_reduced.npy'
path_vocab = ml_path + 'vocab_pretrained_reduced.pkl'
path_train_pos = ml_path + 'pos_train.txt'
path_train_neg = ml_path + 'neg_train.txt'
path_test = ml_path + 'test_data.txt'

# define file names
submission_file = ml_path + 'cnn1_128_reduced_dropout.csv'
network_file = ml_path + 'network.pt'


# load word embeddings
embeddings = np.load(path_embeddings)
# add line of zeroes to the embeddings for empty words
embeddings = np.append(np.zeros((1, embeddings.shape[1])), embeddings, axis=0)
# load vocabulary
with open(path_vocab, 'rb') as f:
    vocab = pickle.load(f)


# find maximal tweet length (number of words)
longest = 0
for file in [path_train_pos, path_train_neg, path_test]:
    with open(file) as f:
        for line in f:
            length = len(line.strip().split())
            if length > longest:
                longest = length          
print("Longest tweet has {:d} words".format(longest))


'''
For each tweet we create an array containing the indexes of the words in
the word embedding matrix. All of these vectors have equal length equivalent
to the number of words in the longest tweet. If a tweet doesn't fill everything,
we pad with 0. this index corresponds to the embedding [0, 0, 0, ... 0, 0].

Data type is int32 in order to minimize memory usage.

For the labels, we use the value 0 for negative tweets and 1 for positive tweets.
'''

#initiate empty feature and label lists
x = []
y = []

# process positive tweets
with open(path_train_pos) as f:
    for line in f:
        tweet = np.int32(np.zeros((longest)))
        wordcount = 0
        y.append(1)
        for word in line.strip().split():
            index = vocab.get(word, -1);
            # skip words for which we have no embedding
            if(index != -1):
                tweet[wordcount] = index + 1
                wordcount += 1
        x.append(tweet)

# process negative tweets
with open(path_train_neg) as f:
    for line in f:
        tweet = np.int32(np.zeros((longest)))
        wordcount = 0
        y.append(0)
        for word in line.strip().split():
            index = vocab.get(word, -1);
            # skip words for which we have no embedding
            if(index != -1):
                tweet[wordcount] = index + 1
                wordcount += 1
        x.append(tweet)

# transform to numpy array
x_train = np.asarray(x)
y_train = np.asarray(y)
 
# Shuffle tweets
x_train, y_train = shuffle(x_train, y_train)


# We proceed process test tweets in the same way as the training tweets
x = []

with open(path_test) as f:
    for line in f:
        tweet = np.int32(np.zeros((longest)))
        wordcount = 0
        # filter out the IDs and first comma
        line_bare = line[(line.index(",")+1):]
        for word in line_bare.strip().split():
            index = vocab.get(word, -1);
            # skip words for which we have no embedding
            if(index != -1):
                tweet[wordcount] = index + 1
                wordcount += 1
        x.append(tweet)

        # convert to numpy array
x_test = np.asarray(x)


# specify the network you want to use
net = SimpleConvNet(torch.from_numpy(embeddings).float()).cuda()
# decide how many training tweets to use for validation
val_prop = 10000
batch_size = 1024
epochs = 10
# defines after how many batches loss and accuracy are displayed
print_every = 50
# choose loss function
criterion = torch.nn.BCELoss()
# choose optimizer (Adam does fine most of the time)
optimizer = torch.optim.Adam(net.parameters())


# cut validation data from training data.
# convert numpy arrays to torch tensors
x_val_torch = torch.from_numpy(x_train[0:val_prop, :])
y_val_torch = torch.from_numpy(y_train[0:val_prop]).float()
x_train_torch = torch.from_numpy(x_train[val_prop + 1:, :])
y_train_torch = torch.from_numpy(y_train[val_prop + 1:]).float()

# create batch loaders
train_set = utils.TensorDataset(x_train_torch, y_train_torch)
train_loader = utils.DataLoader(train_set, batch_size, shuffle=False)
val_set = utils.TensorDataset(x_val_torch, y_val_torch)
val_loader = utils.DataLoader(val_set, batch_size, shuffle=False)

# Run training
steps = 0
running_loss = 0
start = time.time()

for e in range(epochs):
    for tweets, labels in iter(train_loader):
        steps += 1
        # converting to Variable is necessary in order to compute the gradient later
        inputs = Variable(tweets).cuda()
        targets = Variable(labels).cuda()
        # set gradient to zero
        optimizer.zero_grad()
        # forward inputs through the net
        outputs = net.forward(inputs.long())
        # compute loss and gradient
        loss = criterion(outputs, targets)
        loss.backward()
        # update weights
        optimizer.step()
        running_loss += loss.data[0]
        
        if steps % print_every == 0:
            stop = time.time()
            accuracy = 0
            n = 0
            # compute accuracy on validation set
            net.eval()
            for tweets, labels in iter(val_loader):
                predictions = net.predict(tweets.long().cuda()).cpu()
                accuracy += sum(predictions.data.numpy() == labels.data.numpy())
                n += labels.data.numpy().size
            net.train()
            
            print("Epoch {} / {}\t".format(e+1, epochs),
                  "Loss {:.4f}\t".format(running_loss / print_every),
                  "Validation accuracy {:.4f}\t".format(accuracy / n),
                  "{:.4f} s/batch".format((stop - start)/print_every))
            running_loss = 0
            start = time.time()


# Save the trained network
torch.save(net.cpu(), network_file)

# Load a saved network
net = torch.load(network_file).cuda()


# compute accuracy based on a part of the training data
net.eval()
for tweets, labels in iter(val_loader):
    predictions = net.predict(tweets.long().cuda()).cpu()
    accuracy += sum(predictions.data.numpy() == labels.data.numpy())
    n += labels.data.numpy().size
            
print("Accuracy on validation set: {:.4f}".format(accuracy / n))


# Compute preditions and transform the labels to (-1, 1)
net.eval()
test_loader = utils.DataLoader(torch.from_numpy(x_test), batch_size, shuffle = False)
submission_labels = np.zeros((0))
for tweets in iter(test_loader):
    predictions = net.predict(tweets.long().cuda()).cpu()
    labels = predictions.data.numpy() * 2 - 1
    submission_labels = np.concatenate((submission_labels, labels), axis=0)
    
# save predicitons on the test set to csv file.
ids = np.arange(len(submission_labels)) + 1
create_csv_submission(ids, submission_labels, submission_file)
print("Submissions saved as", submission_file)

Longest tweet has 64 words


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:165: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch 1 / 10	 Loss 0.5241	 Validation accuracy 0.7768	 0.0284 s/batch
Epoch 1 / 10	 Loss 0.4165	 Validation accuracy 0.8066	 0.0261 s/batch
Epoch 1 / 10	 Loss 0.3864	 Validation accuracy 0.8122	 0.0264 s/batch
Epoch 2 / 10	 Loss 0.3779	 Validation accuracy 0.8190	 0.0259 s/batch
Epoch 2 / 10	 Loss 0.3638	 Validation accuracy 0.8223	 0.0265 s/batch
Epoch 2 / 10	 Loss 0.3597	 Validation accuracy 0.8266	 0.0264 s/batch
Epoch 2 / 10	 Loss 0.3519	 Validation accuracy 0.8294	 0.0282 s/batch
Epoch 3 / 10	 Loss 0.3497	 Validation accuracy 0.8306	 0.0262 s/batch
Epoch 3 / 10	 Loss 0.3403	 Validation accuracy 0.8298	 0.0262 s/batch
Epoch 3 / 10	 Loss 0.3369	 Validation accuracy 0.8302	 0.0262 s/batch
Epoch 3 / 10	 Loss 0.3322	 Validation accuracy 0.8344	 0.0265 s/batch
Epoch 4 / 10	 Loss 0.3312	 Validation accuracy 0.8325	 0.0260 s/batch
Epoch 4 / 10	 Loss 0.3239	 Validation accuracy 0.8338	 0.0264 s/batch
Epoch 4 / 10	 Loss 0.3211	 Validation accuracy 0.8349	 0.0262 s/batch
Epoch 5 / 10	 Loss 0

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type SimpleConvNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Accuracy on validation set: 0.8368
Submissions saved as /content/gdrive/My Drive/ML2018/cnn1_128_reduced_dropout.csv


In [0]:
# Load a saved network
net = torch.load(network_file).cuda()


# compute accuracy based on a part of the training data
for tweets, labels in iter(val_loader):
    predictions = net.predict(tweets.long().cuda()).cpu()
    accuracy += sum(predictions.data.numpy() == labels.data.numpy())
    n += labels.data.numpy().size
            
print("Accuracy on validation set: {:.4f}".format(accuracy / n))


# Compute preditions and transform the labels to (-1, 1)
test_loader = utils.DataLoader(torch.from_numpy(x_test), batch_size, shuffle = False)
submission_labels = np.zeros((0))
for tweets in iter(test_loader):
    predictions = net.predict(tweets.long().cuda()).cpu()
    labels = predictions.data.numpy() * 2 - 1
    submission_labels = np.concatenate((submission_labels, labels), axis=0)
    
# save predicitons on the test set to csv file.
ids = np.arange(len(submission_labels)) + 1
create_csv_submission(ids, submission_labels, submission_file)
print("Submissions saved as", submission_file)


NameError: ignored